In [1]:
#from eps_utils import *
from typing import *

import numpy as np
import pandas as pd

from  call_bedrock_fast_py import *

In [4]:
pd.read_csv('School_books.csv')

,asin,item_name,gl_product_group,gl_product_group_desc,recommended_browse_node_id1,recommended_browse_node_id2,item_description,product_description,total_ordered_units,Total_Gvs,rbn_eip,bisac_subject_category,thema_classification_code,browse_node_ids_list,sp,branch_validate,leaf_validate
0,B08KGXVK3F,Themes In Indian History Part - 2 For Class - ...,14,gl_book,4149808031,NaN,NaN,NaN,0.0,25.0,4.149808e+09,NaN,NaN,4149808031,['4149808031'],True,False
1,8196168977,Vacation Activity Book Class 1 (A Bridge to Cl...,14,gl_book,4149807031,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,4149807031,['4149807031'],True,False
2,8176139491,CBSE Complete Solutions Sample Papers Geograph...,14,gl_book,4149807031,NaN,NaN,JPH Class 12 Examate Geography Sample Papers C...,0.0,18.0,4.149807e+09,NaN,yp,4149807031,['4149807031'],True,False
3,0733985912,International Mathematics (International Bacca...,14,gl_book,4149838031,4.662559e+09,This is the fifth book in the five book Intern...,NaN,0.0,120.0,4.149838e+09,JUVENILE NONFICTION / Mathematics / General,YPMF,"4149838031,4662559031","['4149838031', '4662559031']",True,False
4,9350951967,Cordova Mastering Grammer for Class 5,14,gl_book,4149808031,4.662554e+09,School textbooks.,NaN,53.0,806.0,4.149808e+09,NaN,NaN,"4149808031,4662554031,4149814031,4662555031","['4149808031', '4662554031', '4149814031', '46...",True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60828,B0CQ585N5M,MA GAYATRI BOOK STORE Junior English Translati...,14,gl_book,4149807031,NaN,NaN,Junior English Translation Book,1.0,17.0,4.149807e+09,NaN,NaN,4149807031,['4149807031'],True,False
60829,9352830881,Self-Learning English Course with Activities -...,14,gl_book,4149808031,4.662554e+09,Self-Learning English Course with Activities i...,NaN,1.0,111.0,4.149808e+09,NaN,NaN,"4149808031,4662554031,4149814031,4662555031","['4149808031', '4662554031', '4149814031', '46...",True,False
60830,2718901713,Liturgie des heures - careme temps pascal - 2,14,gl_book,4149808031,NaN,NaN,NaN,0.0,0.0,4.149808e+09,NaN,NaN,4149808031,['4149808031'],True,False
60831,8130916029,Viva - Albatross Dictionary of Synonyms & Anto...,14,gl_book,4149808031,4.662554e+09,School textbook.,NaN,0.0,4.0,4.149808e+09,NaN,NaN,"4149808031,4662554031","['4149808031', '4662554031']",True,False


In [9]:
entrance_school_prompt_template ='''Human:
<role>
You are an shopping expert who is assessing the categorization of books. Your goal is to determine what is the category of a book given the details of the book.
</role>

<task>

Your task is determine the category of a book which belongs for School Books. The possible categories are mentioned in the <categories></categories> XML tags. You can use the book details given in <book-details></book-details> XML tags to make your decision.
First, use your judgement to think about which category the higher education book belongs to, and output this judgement in <thinking></thinking> XML tags. This is a space for you to write down your thoughts.
Next, output the category you think this book best belongs to in the <category></category> XML tags. This should be copied word-for-word from one of the <item></item> XML tags. If none of the categories match, you should output "NONE OF THE ABOVE" in <category></category> XML tags.
Finally, Mention how confident you are about the predicted category of this book by returning "Very Confident", "Somewhat Confident" or "Not Confident" in <confidence></confidence> XML tags. This should be done after <category></category> XML tags.
Stop responding after </confidence>.
</task>

<categories>
{category}
</categories>


<book-details>
<title>{title}</title>
<description>{description}</description>
</book-details>

Assistant:
<thinking>

'''.strip()

In [10]:
entrance_school_prompt_template1 ='''Human:
<role>
You are an shopping expert who is assessing the categorization of books. Your goal is to determine what is the category of a book given the details of the book.
</role>

<task>

Your task is determine the category of a book which belongs for School Books. The possible categories are mentioned in the <categories></categories> XML tags. You can use the book details given in <product-details></product-details> XML tags to make your decision.
First, use your judgement to think about which category the higher education book belongs to, and output this judgement in <thinking></thinking> XML tags. This is a space for you to write down your thoughts.
Next, output the category you think this book best belongs to in the <category></category> XML tags. This should be copied word-for-word from one of the <item></item> XML tags. If none of the categories match, you should output "NONE OF THE ABOVE" in <category></category> XML tags.
Finally, Mention how confident you are about the predicted category of this book by returning "Very Confident", "Somewhat Confident" or "Not Confident" in <confidence></confidence> XML tags. This should be done after <category></category> XML tags.
Stop responding after </confidence>.
</task>

<categories>
{category}
</categories>

<book-details>
<title>{title}</title>
<description>{description}</description>
<root>{root}</root>
</book-details>



Assistant:
<thinking>

'''.strip()

In [3]:
volt_file=pd.read_csv('44571_mapping(2).txt',sep='\t')
volt_file_hig=volt_file[volt_file['Named path'].str.contains('\[Books, School Books')]
volt_file_hig['Named path'].replace(['\[Books, School Books, ','\]'],"",inplace=True,regex=True)
volt_file_hig=volt_file_hig[['Named path','Classification Key','Is Leaf']]
volt_file_hig.reset_index(drop=True,inplace=True)
volt_file_hig
volt_file_hig['root_node']=volt_file_hig['Named path'].str.split(',',expand=True)[0]

volt_file_hig['root']='<item>'+volt_file_hig['root_node']+'</item>'

root_category="\n".join(list(volt_file_hig['root'].drop_duplicates()))
root_category

/tmp/ipykernel_468/3325485461.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  volt_file_hig['Named path'].replace(['\[Books, School Books, ','\]'],"",inplace=True,regex=True)
/tmp/ipykernel_468/3325485461.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volt_file_hig['Named path'].replace(['\[Books, School Books, ','\]'],"",inplace=True

'<item>CBSE</item>\n<item>Olympiads & Scholarship Exams</item>\n<item>State Education Boards</item>\n<item>CISCE</item>\n<item>School Atlases</item>\n<item>International School Boards</item>\n<item>Open Schooling</item>\n<item>School Bundles</item>\n<item>[Books</item>'

In [4]:
import pandas as pd
volt_file=pd.read_csv('44571_mapping(2).txt',sep='\t')
volt_file_hig=volt_file[volt_file['Named path'].str.contains('\[Books, School Books')]
volt_file_hig['Named path'].replace(['\[Books, School Books, ','\[','\]'],"",inplace=True,regex=True)
volt_file_hig['root']='<item>'+volt_file_hig['Named path'].str.split(',',expand=True)[0]+'</item>'
volt_file_hig=volt_file_hig[['Named path','Classification Key','root','Is Leaf']]

#volt_file_hig['Leaf Path']=np.where(volt_file_hig['Named path'].str.contains('Engineering Textbooks')==True,volt_file_hig['Named path'],"")
#volt_file_hig_part.replace('Engineering Textbooks, ',"",inplace=True,regex=True)
ass=volt_file_hig['Named path'].str.split(', ',expand=True).drop(columns=0)
volt_file_hig['leaf_1']='<item>'+ass.stack().groupby(level=0).agg(", ".join)+'</item>'
volt_file_hig['Named path2']='<item>'+volt_file_hig['Named path']+'</item>'
volt_file_hig.reset_index(drop=True,inplace=True)
volt_file_hig


volt_file_hig['Named path2']
leaf_category="\n".join(list(volt_file_hig['Named path2'].drop_duplicates()))


/tmp/ipykernel_468/142448783.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  volt_file_hig['Named path'].replace(['\[Books, School Books, ','\[','\]'],"",inplace=True,regex=True)
/tmp/ipykernel_468/142448783.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volt_file_hig['Named path'].replace(['\[Books, School Books, ','\[','\]'],"",inpl

In [5]:
from contextlib import contextmanager
from typing import *
@contextmanager
def pd_display(
        max_rows: Optional[int] = None,
        max_cols: Optional[int] = None,
        max_colwidth: Optional[int] = None,
        vertical_align: str = 'top',
        text_align: str = 'left',
        ignore_css: bool = False,
):
    try:
        from IPython.display import display
    except ImportError:
        display = print

    def disp(df: pd.DataFrame):
        css = [
            ## Align header to center
            {
                'selector': 'th',
                'props': [
                    ('vertical-align', 'center'),
                    ('text-align', 'center'),
                    ('padding', '10px'),
                ]
            },
            ## Align cell to top and left
            {
                'selector': 'td',
                'props': [
                    ('vertical-align', vertical_align),
                    ('text-align', text_align),
                    ('padding', '10px'),
                ]
            },

        ]
        if not ignore_css and isinstance(df, pd.DataFrame):
            df = df.style.set_table_styles(css)
        display(df)

    with pd.option_context(
            'display.max_rows', max_rows,
            'display.max_columns', max_cols,
            'max_colwidth', max_colwidth,
            'display.expand_frame_repr', False,
    ):
        yield disp

In [7]:
aud=pd.read_csv('School_books.csv')
aud['description']=np.where(aud['product_description'].isnull()==True,aud['item_description'],aud['product_description'])

In [11]:
def create_root_prompt(row):
    return entrance_school_prompt_template.format(category=root_category,title=row['item_name'],description=row['description'])
    

aud['root_prompt']=aud.apply(create_root_prompt,axis=1)
#aud=aud.head(2000)
aud.reset_index(drop=True,inplace=True)
#aud['prompt']=entrance_exam_prompt_template1.format(
# with pd_display() as disp:
#     disp(aud.sample(3))
display(aud)
print(aud.loc[0,'root_prompt'])

,asin,item_name,gl_product_group,gl_product_group_desc,recommended_browse_node_id1,recommended_browse_node_id2,item_description,product_description,total_ordered_units,Total_Gvs,rbn_eip,bisac_subject_category,thema_classification_code,browse_node_ids_list,sp,branch_validate,leaf_validate,description,root_prompt
0,B08KGXVK3F,Themes In Indian History Part - 2 For Class - ...,14,gl_book,4149808031,NaN,NaN,NaN,0.0,25.0,4.149808e+09,NaN,NaN,4149808031,['4149808031'],True,False,NaN,Human:\n<role>\nYou are an shopping expert who...
1,8196168977,Vacation Activity Book Class 1 (A Bridge to Cl...,14,gl_book,4149807031,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,4149807031,['4149807031'],True,False,NaN,Human:\n<role>\nYou are an shopping expert who...
2,8176139491,CBSE Complete Solutions Sample Papers Geograph...,14,gl_book,4149807031,NaN,NaN,JPH Class 12 Examate Geography Sample Papers C...,0.0,18.0,4.149807e+09,NaN,yp,4149807031,['4149807031'],True,False,JPH Class 12 Examate Geography Sample Papers C...,Human:\n<role>\nYou are an shopping expert who...
3,0733985912,International Mathematics (International Bacca...,14,gl_book,4149838031,4.662559e+09,This is the fifth book in the five book Intern...,NaN,0.0,120.0,4.149838e+09,JUVENILE NONFICTION / Mathematics / General,YPMF,"4149838031,4662559031","['4149838031', '4662559031']",True,False,This is the fifth book in the five book Intern...,Human:\n<role>\nYou are an shopping expert who...
4,9350951967,Cordova Mastering Grammer for Class 5,14,gl_book,4149808031,4.662554e+09,School textbooks.,NaN,53.0,806.0,4.149808e+09,NaN,NaN,"4149808031,4662554031,4149814031,4662555031","['4149808031', '4662554031', '4149814031', '46...",True,False,School textbooks.,Human:\n<role>\nYou are an shopping expert who...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60828,B0CQ585N5M,MA GAYATRI BOOK STORE Junior English Translati...,14,gl_book,4149807031,NaN,NaN,Junior English Translation Book,1.0,17.0,4.149807e+09,NaN,NaN,4149807031,['4149807031'],True,False,Junior English Translation Book,Human:\n<role>\nYou are an shopping expert who...
60829,9352830881,Self-Learning English Course with Activities -...,14,gl_book,4149808031,4.662554e+09,Self-Learning English Course with Activities i...,NaN,1.0,111.0,4.149808e+09,NaN,NaN,"4149808031,4662554031,4149814031,4662555031","['4149808031', '4662554031', '4149814031', '46...",True,False,Self-Learning English Course with Activities i...,Human:\n<role>\nYou are an shopping expert who...
60830,2718901713,Liturgie des heures - careme temps pascal - 2,14,gl_book,4149808031,NaN,NaN,NaN,0.0,0.0,4.149808e+09,NaN,NaN,4149808031,['4149808031'],True,False,NaN,Human:\n<role>\nYou are an shopping expert who...
60831,8130916029,Viva - Albatross Dictionary of Synonyms & Anto...,14,gl_book,4149808031,4.662554e+09,School textbook.,NaN,0.0,4.0,4.149808e+09,NaN,NaN,"4149808031,4662554031","['4149808031', '4662554031']",True,False,School textbook.,Human:\n<role>\nYou are an shopping expert who...


Human:
<role>
You are an shopping expert who is assessing the categorization of books. Your goal is to determine what is the category of a book given the details of the book.
</role>

<task>

Your task is determine the category of a book which belongs for School Books. The possible categories are mentioned in the <categories></categories> XML tags. You can use the book details given in <book-details></book-details> XML tags to make your decision.
First, use your judgement to think about which category the higher education book belongs to, and output this judgement in <thinking></thinking> XML tags. This is a space for you to write down your thoughts.
Next, output the category you think this book best belongs to in the <category></category> XML tags. This should be copied word-for-word from one of the <item></item> XML tags. If none of the categories match, you should output "NONE OF THE ABOVE" in <category></category> XML tags.
Finally, Mention how confident you are about the predicted

In [19]:
aud.reset_index(drop=True,inplace=True)
aud.fillna("",inplace=True)

/tmp/ipykernel_31811/2540733322.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  aud.fillna("",inplace=True)


In [ ]:
model_name='anthropic.claude-3-sonnet-20240229-v1:0'
aud['root_output']=call_bedrock_fast(prompts=aud['root_prompt'], model=model_name,
                # model='anthropic.claude-instant-v1',
                max_new_tokens=500,
                temperature=0.1)

  0%|                                                                     | 0/60833 [00:00<?, ?it/s]